In [1]:
# 查看版本信息
import sys
print(sys.version)
# !pip list

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


In [1]:
pip list

Package                        Version
------------------------------ ---------------
absl-py                        0.8.1
aiofiles                       23.2.1
aiohttp                        3.8.3
aiosignal                      1.3.1
alembic                        1.8.1
altair                         4.2.0
annotated-types                0.5.0
anyio                          3.7.1
argon2-cffi                    21.3.0
argon2-cffi-bindings           21.2.0
aspy.yaml                      1.3.0
astor                          0.8.1
astroid                        2.4.1
async-generator                1.10
async-timeout                  4.0.2
asynctest                      0.13.0
attrs                          22.1.0
audioread                      2.1.8
autopep8                       1.6.0
Babel                          2.8.0
backcall                       0.1.0
backports.zoneinfo             0.2.1
bce-python-sdk                 0.8.53
beautifulsoup4                 4.11.1
bleach              

In [2]:
import os
import paddle
import paddlenlp
from paddle.io import Dataset
from paddlenlp.datasets import load_dataset,MapDataset
import pandas as pd

In [3]:
# 训练类
class MyTrainDataset(Dataset):
    def __init__(self, file):
        def read_train(file):
            """Reads data"""
            f = pd.read_csv(file, sep="\t")
            head = None
            for index, row in f.iterrows():
                text = row.get('text_a')
                label = row.get('label')
                if text is not None and label is not None:
                    yield {"text": text, "label": label, "qid": ''}
        self.data = list(read_train(file))

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

    def get_labels(self):
        return ["0", "1", "2"]

train = MyTrainDataset('data/train.csv')
train_ds = MapDataset(train,label_list=train.get_labels())

In [4]:
# 开发类
class MyDevDataset(Dataset):
    def __init__(self, file):
        def read_dev(file):
            """Reads data"""
            f = pd.read_csv(file, sep="\t")
            head = None
            for index, row in f.iterrows():
                text = row.get('text_a')
                label = row.get('label')
                qid = row.get('qid')
                if text is not None and label is not None and qid is not None:
                    yield {"text": text, "label": label, "qid": qid}
        self.data = list(read_dev(file))

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

    def get_labels(self):
        return ["0", "1", "2"]

dev = MyDevDataset('data/dev.csv')
dev_ds = MapDataset(dev,label_list=dev.get_labels())

In [5]:
# 测试类
class MyTestDataset(Dataset):
    def __init__(self, file):
        def read_test(file):
            """Reads data"""
            f = pd.read_csv(file, sep="\t")
            head = None
            for index, row in f.iterrows():
                text = row.get('text_a')
                qid = row.get('qid')
                if text is not None and qid is not None:
                    yield {"text": text, "label": '', "qid": qid}
        self.data = list(read_test(file))

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

    def get_labels(self):
        return ["0", "1", "2"]

test = MyTestDataset('data/test.csv')
test_ds = MapDataset(test,label_list=test.get_labels())

In [23]:
#加载数据集,弃用
# from paddle.io import Dataset
# from paddlenlp.datasets import load_dataset,MapDataset
# import pandas as pd

# def read_train(filename):
#     """Reads data"""
#     f = pd.read_csv(filename, sep="\t")
#     head = None
#     for index, row in f.iterrows():
#         text = row.get('text_a')
#         label = row.get('label')
#         if text is not None and label is not None:
#             yield {"text": text, "label": label, "qid": ''}

# def read_dev(filename):
#     """Reads data"""
#     f=pd.read_csv(filename,sep="\t")
#     head = None
#     for index, row in f.iterrows():
#         text = row.get('text_a')
#         label = row.get('label')
#         qid = row.get('qid')
#         if text is not None and label is not None and qid is not None:
#             yield {"text": text, "label": label, "qid": qid}

# def read_test(filename):
#     """Reads data"""
#     f=pd.read_csv(filename,sep="\t")
#     head = None
#     for index, row in f.iterrows():
#         text = row.get('text_a')
#         qid = row.get('qid')
#         if text is not None and qid is not None:
#             yield {"text": text, "label": '', "qid": qid}


# # 使用你的 read 函数读取数据
# train = list(read_train("data/train.csv"))
# dev = list(read_dev("data/dev.csv"))
# test = list(read_test("data/test.csv"))

# # 创建数据集对象
# train_ds = MapDataset(train)
# dev_ds = MapDataset(dev)
# test_ds = MapDataset(test)

# print(type(train_ds))
# print(type(dev_ds))
# print(type(test_ds))

<class 'paddlenlp.datasets.dataset.MapDataset'>
<class 'paddlenlp.datasets.dataset.MapDataset'>
<class 'paddlenlp.datasets.dataset.MapDataset'>


In [6]:
# 数据集返回为MapDataset类型，查看数据样本详情
print("数据类型:", type(train_ds))
# label代表标签，qid代表数据编号，测试集中不包含标签信息
print("训练集样例:", train_ds[0])
print("验证集样例:", dev_ds[0])
print("测试集样例:", test_ds[0])

数据类型: <class 'paddlenlp.datasets.dataset.MapDataset'>
训练集样例: {'text': '今早来吃个早餐，体验很差！图片的炒蛋很少，而且放很久，面包是不怎么热的，跟平时吃得不一样颜色，那块鸡肉好黑的样子！拿餐后发现好多餐盘没有收拾，无从下坐，叫经理换鸡蛋就真的换了鸡蛋，而且餐里面加了油条，油条感觉不是新鲜炸的不怎么热，像放很久，只有薯饼是热和脆！唉', 'label': 2, 'qid': ''}
验证集样例: {'text': '位置十分好，就在出站口，服务态度也非常不错，还是挺推荐的', 'label': 0, 'qid': 0}
测试集样例: {'text': '今日Mc Cafe 免费赠饮拿铁，好多人，排队排长龙，龙尾差唔多去到手扶电梯口，睇来大家都好眼训 虽然人好多，但冲咖啡既姐姐一d都唔马虎，奶泡刚打出来继续系噔一噔，慢慢拉花式撞入咖啡，绝对无因为人多而求其贪快，赞！', 'label': '', 'qid': 0}


In [7]:
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

#定义"ernie-3.0-medium-zh"模型
model_name = "ernie-3.0-medium-zh"

#加载预训练模型，并指定分类的类别数为训练集的标签数量
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=len(train_ds.label_list))

#加载预训练模型对应的分词器。
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2024-03-27 22:11:02,064] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-3.0-medium-zh'.
[2024-03-27 22:11:02,068] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-3.0-medium-zh
[2024-03-27 22:11:02,071] [    INFO] - Downloading ernie_3.0_medium_zh.pdparams from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh.pdparams
100%|██████████| 313M/313M [00:08<00:00, 38.9MB/s] 
W0327 22:11:10.612514   173 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W0327 22:11:10.618561   173 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
[2024-03-27 22:11:13,908] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'ernie-3.0-medium-zh'.
[2024-03-27 22:11:13,

In [8]:
import functools
import numpy as np
from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# 数据预处理函数，利用分词器将文本转化为整数序列
# examples：输入的数据样本，通常是一个字典，包含了文本数据和对应的标签。
# tokenizer：分词器，用于将文本数据转换为模型可以接受的格式。
# max_seq_length：序列的最大长度，超过这个长度的序列将被截断。
# is_test：一个布尔值，表示当前是否为测试模式。
def preprocess_function(examples, tokenizer, max_seq_length, is_test=False):

    # 使用分词器tokenizer对输入的文本examples["text"]进行处理，得到的结果保存在result中。
    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)

    # 如果当前不是测试模式（is_test=False），那么将输入数据的标签examples["label"]添加到result中
    if not is_test:
        result["labels"] = examples["label"]
    return result

# 这个函数等同于调用preprocess_function
trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128)
# 这两行代码将trans_func函数应用到训练集和开发集的每个样本上。
# 这意味着每个样本都会被分词，序列长度会被限制在128以内，如果不是测试模式，还会添加标签。
train_ds = train_ds.map(trans_func)
dev_ds = dev_ds.map(trans_func)

# collate_fn函数构造，将不同长度序列充到批中数据的最大长度，再将数据堆叠
collate_fn = DataCollatorWithPadding(tokenizer)

# 定义BatchSampler，选择批大小和是否随机乱序，进行DataLoader
# 这两行代码定义了批量采样器（BatchSampler）。批量采样器用于从数据集中选择批量的数据。
# 对于训练集，批大小被设置为32，数据会被随机打乱；对于开发集，批大小被设置为64，数据不会被打乱。
train_batch_sampler = BatchSampler(train_ds, batch_size=32, shuffle=True)
dev_batch_sampler = BatchSampler(dev_ds, batch_size=64, shuffle=False)
# 这两行代码创建了数据加载器。数据加载器用于在训练和评估模型时批量加载数据。
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
dev_data_loader = DataLoader(dataset=dev_ds, batch_sampler=dev_batch_sampler, collate_fn=collate_fn)

In [9]:
# Adam优化器、交叉熵损失函数、accuracy评价指标
# 创建了一个AdamW优化器，用于优化模型的参数。这里的学习率设置为2e-5，优化的参数是模型的参数
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())
# 这行代码定义了交叉熵损失函数，这是一种常用的分类任务的损失函数
criterion = paddle.nn.loss.CrossEntropyLoss()
# 定义了准确率作为评估指标，用于评估模型的性能
metric = paddle.metric.Accuracy()

In [10]:
# 开始训练
import time
import paddle.nn.functional as F

from eval import evaluate

epochs = 5 # 训练轮次
ckpt_dir = "new_ernie_ckpt" # 训练过程中保存模型参数的文件夹
best_acc = 0    # 用于在训练过程中记录最好的模型性能
best_step = 0   # 用于在训练过程中记录最好的模型性能对应的步骤数
global_step = 0 # 迭代次数
tic_train = time.time()
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值、分类概率值、准确率
        logits = model(input_ids, token_type_ids)   # 通过模型获取预测结果（logits）
        loss = criterion(logits, labels)    # 使用交叉熵损失函数（criterion）计算预测结果（logits）和真实标签（labels）之间的损失。
        probs = F.softmax(logits, axis=1)   # 使用softmax函数计算logits的概率分布，axis=1表示概率计算是在每个样本的输出类别上进行的。
        correct = metric.compute(probs, labels) # 计算预测的准确性，metric.compute根据概率（probs）和真实标签（labels）来计算正确预测的数量。
        metric.update(correct)              # 更新评估指标，将当前批次的正确预测数量添加到累积的评估中。
        acc = metric.accumulate()           # 累积评估指标，计算到目前为止的总准确率。

        # 每迭代10次，打印损失函数值、准确率、计算速度
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传，更新参数
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        # 每迭代100次，评估当前训练的模型、保存当前模型参数和分词器的词表等
        if global_step % 100 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            print(global_step, end=' ')
            acc_eval = evaluate(model, criterion, metric, dev_data_loader)
            if acc_eval > best_acc:
                best_acc = acc_eval
                best_step = global_step

                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

global step 10, epoch: 1, batch: 10, loss: 0.80555, accu: 0.53438, speed: 4.72 step/s
global step 20, epoch: 1, batch: 20, loss: 0.84935, accu: 0.55937, speed: 9.78 step/s
global step 30, epoch: 1, batch: 30, loss: 0.70245, accu: 0.59271, speed: 9.81 step/s
global step 40, epoch: 1, batch: 40, loss: 0.87151, accu: 0.62109, speed: 9.73 step/s
global step 50, epoch: 1, batch: 50, loss: 0.52292, accu: 0.64625, speed: 9.74 step/s
global step 60, epoch: 1, batch: 60, loss: 0.61718, accu: 0.65781, speed: 9.73 step/s
global step 70, epoch: 1, batch: 70, loss: 0.66179, accu: 0.66071, speed: 9.65 step/s
global step 80, epoch: 1, batch: 80, loss: 0.63690, accu: 0.67109, speed: 9.74 step/s
global step 90, epoch: 1, batch: 90, loss: 0.43375, accu: 0.68021, speed: 9.63 step/s
global step 100, epoch: 1, batch: 100, loss: 0.48800, accu: 0.69000, speed: 9.54 step/s
100 eval loss: 0.60693, accuracy: 0.72490


[2024-03-27 22:12:18,701] [    INFO] - tokenizer config file saved in new_ernie_ckpt/tokenizer_config.json
[2024-03-27 22:12:18,706] [    INFO] - Special tokens file saved in new_ernie_ckpt/special_tokens_map.json


global step 110, epoch: 1, batch: 110, loss: 0.61144, accu: 0.75000, speed: 1.05 step/s
global step 120, epoch: 1, batch: 120, loss: 0.49467, accu: 0.74375, speed: 9.55 step/s
global step 130, epoch: 1, batch: 130, loss: 0.62742, accu: 0.74479, speed: 9.36 step/s
global step 140, epoch: 1, batch: 140, loss: 0.53298, accu: 0.75391, speed: 9.03 step/s
global step 150, epoch: 1, batch: 150, loss: 0.59486, accu: 0.74562, speed: 8.64 step/s
global step 160, epoch: 1, batch: 160, loss: 0.63068, accu: 0.73906, speed: 9.69 step/s
global step 170, epoch: 1, batch: 170, loss: 0.60973, accu: 0.73438, speed: 9.00 step/s
global step 180, epoch: 1, batch: 180, loss: 0.67303, accu: 0.73984, speed: 9.50 step/s
global step 190, epoch: 1, batch: 190, loss: 0.65287, accu: 0.73438, speed: 9.70 step/s
global step 200, epoch: 1, batch: 200, loss: 0.68225, accu: 0.73188, speed: 9.57 step/s
200 eval loss: 0.61182, accuracy: 0.73752


[2024-03-27 22:12:40,465] [    INFO] - tokenizer config file saved in new_ernie_ckpt/tokenizer_config.json
[2024-03-27 22:12:40,469] [    INFO] - Special tokens file saved in new_ernie_ckpt/special_tokens_map.json


global step 210, epoch: 1, batch: 210, loss: 0.80368, accu: 0.76875, speed: 0.83 step/s
global step 220, epoch: 1, batch: 220, loss: 0.45309, accu: 0.76250, speed: 9.70 step/s
global step 230, epoch: 1, batch: 230, loss: 0.43064, accu: 0.74167, speed: 9.64 step/s
global step 240, epoch: 1, batch: 240, loss: 0.56690, accu: 0.73828, speed: 9.68 step/s
global step 250, epoch: 1, batch: 250, loss: 0.50859, accu: 0.74000, speed: 9.38 step/s
global step 260, epoch: 1, batch: 260, loss: 0.42541, accu: 0.75156, speed: 9.52 step/s
global step 270, epoch: 1, batch: 270, loss: 0.64059, accu: 0.74375, speed: 9.62 step/s
global step 280, epoch: 1, batch: 280, loss: 0.63037, accu: 0.74141, speed: 9.67 step/s
global step 290, epoch: 1, batch: 290, loss: 0.57337, accu: 0.73993, speed: 9.68 step/s
global step 300, epoch: 1, batch: 300, loss: 0.56406, accu: 0.74125, speed: 9.64 step/s
300 eval loss: 0.58541, accuracy: 0.74173


...

global step 910, epoch: 2, batch: 18, loss: 0.43813, accu: 0.76562, speed: 0.89 step/s
global step 920, epoch: 2, batch: 28, loss: 0.46779, accu: 0.77031, speed: 9.75 step/s
global step 930, epoch: 2, batch: 38, loss: 0.41453, accu: 0.77083, speed: 9.26 step/s
global step 940, epoch: 2, batch: 48, loss: 0.46373, accu: 0.77656, speed: 8.70 step/s
global step 950, epoch: 2, batch: 58, loss: 0.59300, accu: 0.78125, speed: 7.97 step/s
global step 960, epoch: 2, batch: 68, loss: 0.63839, accu: 0.77500, speed: 9.69 step/s
global step 970, epoch: 2, batch: 78, loss: 0.57960, accu: 0.77589, speed: 9.68 step/s
global step 980, epoch: 2, batch: 88, loss: 0.49007, accu: 0.77773, speed: 9.69 step/s
global step 990, epoch: 2, batch: 98, loss: 0.45005, accu: 0.77743, speed: 9.68 step/s
global step 1000, epoch: 2, batch: 108, loss: 0.80670, accu: 0.77719, speed: 9.54 step/s
1000 eval loss: 0.56588, accuracy: 0.74930
global step 1010, epoch: 2, batch: 118, loss: 0.50203, accu: 0.77500, speed: 1.16 ste

[2024-03-27 22:17:44,082] [    INFO] - tokenizer config file saved in new_ernie_ckpt/tokenizer_config.json
[2024-03-27 22:17:44,098] [    INFO] - Special tokens file saved in new_ernie_ckpt/special_tokens_map.json


global step 1810, epoch: 3, batch: 26, loss: 0.59608, accu: 0.82500, speed: 0.84 step/s
global step 1820, epoch: 3, batch: 36, loss: 0.53110, accu: 0.80469, speed: 9.58 step/s
global step 1830, epoch: 3, batch: 46, loss: 0.31444, accu: 0.80208, speed: 9.59 step/s
global step 1840, epoch: 3, batch: 56, loss: 0.36550, accu: 0.79297, speed: 9.60 step/s
global step 1850, epoch: 3, batch: 66, loss: 0.39227, accu: 0.79812, speed: 9.60 step/s
global step 1860, epoch: 3, batch: 76, loss: 0.52328, accu: 0.80208, speed: 9.10 step/s
global step 1870, epoch: 3, batch: 86, loss: 0.35861, accu: 0.80268, speed: 9.56 step/s
global step 1880, epoch: 3, batch: 96, loss: 0.35141, accu: 0.80547, speed: 9.57 step/s
global step 1890, epoch: 3, batch: 106, loss: 0.42680, accu: 0.80694, speed: 9.57 step/s
global step 1900, epoch: 3, batch: 116, loss: 0.91381, accu: 0.80500, speed: 9.57 step/s
1900 eval loss: 0.60727, accuracy: 0.74004
global step 1910, epoch: 3, batch: 126, loss: 0.31703, accu: 0.82500, speed

[2024-03-27 22:20:29,833] [    INFO] - tokenizer config file saved in new_ernie_ckpt/tokenizer_config.json
[2024-03-27 22:20:29,836] [    INFO] - Special tokens file saved in new_ernie_ckpt/special_tokens_map.json


global step 2710, epoch: 4, batch: 34, loss: 0.56641, accu: 0.82500, speed: 0.87 step/s
global step 2720, epoch: 4, batch: 44, loss: 0.41351, accu: 0.83281, speed: 9.57 step/s
global step 2730, epoch: 4, batch: 54, loss: 0.41019, accu: 0.83750, speed: 9.52 step/s
global step 2740, epoch: 4, batch: 64, loss: 0.28182, accu: 0.85000, speed: 9.52 step/s
global step 2750, epoch: 4, batch: 74, loss: 0.28908, accu: 0.84937, speed: 9.58 step/s
global step 2760, epoch: 4, batch: 84, loss: 0.32912, accu: 0.85260, speed: 9.55 step/s
global step 2770, epoch: 4, batch: 94, loss: 0.47178, accu: 0.84821, speed: 9.58 step/s
global step 2780, epoch: 4, batch: 104, loss: 0.31442, accu: 0.85156, speed: 9.18 step/s
global step 2790, epoch: 4, batch: 114, loss: 0.31457, accu: 0.85243, speed: 9.52 step/s
global step 2800, epoch: 4, batch: 124, loss: 0.31972, accu: 0.84875, speed: 9.59 step/s
2800 eval loss: 0.64816, accuracy: 0.75070
global step 2810, epoch: 4, batch: 134, loss: 0.25362, accu: 0.85000, spee

In [11]:
from eval import evaluate

# 加载ERNIR 3.0最佳模型参数,本地训练模型
params_path = 'new_ernie_ckpt/model_state.pdparams'#定义了预训练模型参数的路径
state_dict = paddle.load(params_path)#这行代码使用PaddlePaddle的load函数从指定路径加载模型参数，并将其存储在state_dict变量中
model.set_dict(state_dict)#将加载的模型参数设置到当前模型中

# 也可以选择加载预先训练好的模型参数结果查看模型训练结果
# model.set_dict(paddle.load('ernie_ckpt_trained/model_state.pdparams'))

print('ERNIE 3.0-Medium 在MCD的dev集表现', end=' ')
# 这行代码是在开发集（dev_data_loader）上评估模型的性能
eval_acc = evaluate(model, criterion, metric, dev_data_loader)

ERNIE 3.0-Medium 在MCD的dev集表现 eval loss: 0.64031, accuracy: 0.75743


In [12]:
# 测试集数据预处理，利用分词器将文本转化为整数序列
trans_func_test = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128, is_test=True)
test_ds_trans = test_ds.map(trans_func_test)

# 进行采样组batch
# 用于将不同长度的序列填充到批中数据的最大长度，然后将数据堆叠起来。这个函数需要一个分词器tokenizer作为输入
collate_fn_test = DataCollatorWithPadding(tokenizer)

# 批量采样器用于从数据集中选择批量的数据。对于测试集，批大小被设置为32，数据不会被打乱。
test_batch_sampler = BatchSampler(test_ds_trans, batch_size=32, shuffle=False)

# 数据加载器用于在进行模型预测时批量加载数据。
test_data_loader = DataLoader(dataset=test_ds_trans, batch_sampler=test_batch_sampler, collate_fn=collate_fn_test)

In [13]:
import paddle
from paddlenlp.transformers import AutoTokenizer, AutoModelForSequenceClassification

# 定义标签映射
label_map = {0: '积极', 1: '中性', 2: '消极'}

def predict_sentiment(text):
    # 使用分词器处理输入文本
    inputs = tokenizer(text, return_tensors='pd', truncation=True, max_length=128, pad_to_max_length=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']

    # 模型预测
    model.eval()
    with paddle.no_grad():
        logits = model(input_ids, token_type_ids)
        probs = F.softmax(logits, axis=1)
        idx = paddle.argmax(probs, axis=1).numpy()
        idx = idx.tolist()

    # 返回预测的情感标签
    return [label_map[i] for i in idx]

# 测试
text = "今天那个负责出餐的女的，跟没吃饭似的，特别的慢腾腾，显示屏显示出餐，走过去问她，语气非常冷漠说等下，到了她叫号，过去也没出餐，过一会儿，出餐了，她跟没啥事似的，也不把餐端出来给我，我就问她好了吧，她才慢悠悠，是非常慢悠悠过去端给我。是不是身体有问题？身体有病就在家休息，别来上班，一点服务意识也没有，慢得跟个鬼似的。！"
print(predict_sentiment(text))

['消极']


In [14]:
# 模型预测分类结果
import paddle.nn.functional as F

#定义了一个标签映射
label_map = {0: '积极', 1: '中性', 2: '消极'}
results = []
#这行代码将模型设置为评估模式。在评估模式下，模型的某些特性，如Dropout和BatchNorm，会被关闭，以确保模型的行为在预测时与训练时不同。
model.eval()

#循环的次数等于测试集的批次数量
for batch in test_data_loader:
    #这行代码从批次数据中提取出输入的token id（input_ids）和token类型id（token_type_ids）
    input_ids, token_type_ids = batch['input_ids'], batch['token_type_ids']

    #通过模型获取预测结果（logits）
    logits = model(batch['input_ids'], batch['token_type_ids'])

    #使用softmax函数计算logits的概率分布
    probs = F.softmax(logits, axis=-1)

    #这行代码找出每个样本预测概率最大的类别的索引
    idx = paddle.argmax(probs, axis=1).numpy()

    #这行代码将索引的numpy数组转换为列表
    idx = idx.tolist()

    #这行代码将数字标签转换为具体的类别
    preds = [label_map[i] for i in idx]

    #这行代码将预测的结果添加到results列表中
    results.extend(preds)

In [18]:
# 存储预测结果  
test = MyTestDataset('data/test.csv')
test_ds = MapDataset(test,label_list=test.get_labels())

res_dir = "./results"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
with open(os.path.join(res_dir, "MCD.tsv"), 'w', encoding="utf8") as f:
    f.write("qid\ttext\tprediction\n")
    for i, pred in enumerate(results):
        f.write(str(test_ds[i]['qid'])+"\t"+test_ds[i]['text']+"\t"+str(pred)+"\n")

In [3]:
'''
预测代码接口
'''
import paddle
from paddlenlp.transformers import AutoTokenizer, AutoModelForSequenceClassification
import paddle.nn.functional as F

# 定义模型
model_name = "new_ernie_ckpt"

# 加载训练模型，指定分类的类别数为训练集的标签数量
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 加载训练模型对应的分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 定义标签映射
label_map = {0: '积极', 1: '中性', 2: '消极'}

def predict_sentiment(text):
    # 使用分词器处理输入文本
    inputs = tokenizer(text, return_tensors='pd', truncation=True, max_length=128, pad_to_max_length=True)
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']

    # 模型预测
    model.eval()
    with paddle.no_grad():
        logits = model(input_ids, token_type_ids)
        probs = F.softmax(logits, axis=1)
        idx = paddle.argmax(probs, axis=1).numpy()
        idx = idx.tolist()

    # 返回预测的情感标签
    return [label_map[i] for i in idx]

# 测试
text = "不知道吃什么，但是麦乐送也挺香的，免去了排队的时间"
print(predict_sentiment(text))

[2024-04-07 15:01:31,798] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'new_ernie_ckpt'.
[2024-04-07 15:01:40,591] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'new_ernie_ckpt'.


['积极']


In [14]:
# 测试
text = "不知道吃什么，但是麦乐送也挺香的，免去了排队的时间"
print(predict_sentiment(text))


['积极']
